<a href="https://colab.research.google.com/github/CodedByAravind/Multi-Turn-Intent-Classification/blob/main/Multi_Turn_Intent_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `gemma` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credent

In [ ]:
import json
import pandas as pd
import re
from transformers import pipeline
import torch

In [ ]:
def clean(text):
    text = text.lower()
    out = []
    for c in text:
        if (c.isalpha() or c.isdigit() or c == " ") and ord(c) < 128:
            out.append(c)
    return "".join(out)


In [ ]:
def preprocess(msg, n=5):
    cleaned_msg = msg[-n:]
    out = []

    for msg in cleaned_msg:
        sender = msg["sender"].lower()
        text = clean(msg["text"])
        out.append(f"{sender}:{text}")

    return " ".join(out)

In [ ]:
pipe = pipeline(
    task="text-generation",
    model="google/gemma-2-2b-it",
    dtype=torch.bfloat16
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def prompt_geneator(conv):
  intents = ["Book Appointment", "Product Inquiry", "Pricing Negotiation", "Support Request", "Follow-Up"]
  prompt = f"""
  You are an AI assistant that classifies conversation intents.
  Possible intents: {', '.join(intents)}.
  Conversation:
  {conv}

  1. Respond with the MOST RELEVANT intent name.
  2. Explain briefly the reason why you chose that intent in 1 short sentence.Only state the reason directly,

  Respond ONLY in the following format and nothing else:
  Intent: <intent name>
  Rationale: <explanation>
  """
  return prompt


In [ ]:
from tqdm import tqdm
def predictor(input_file, batch_size = 8):
  with open(input_file, "r", encoding="utf-8") as f:
    conversations = json.load(f)

  conv_ids = []
  prompts = []
  for conv in conversations:
    conv_id = conv["conversation_id"]
    messages = conv["messages"]
    cleaned = preprocess(messages, n=5)
    prompt = prompt_geneator(cleaned)
    conv_ids.append(conv_id)
    prompts.append(prompt)

  out = []
  for i in tqdm(range(0, len(prompts), batch_size)):
    batch = prompts[i:i + batch_size]
    batch_outputs = pipe(batch, max_new_tokens=100)
    j = i
    for k in range(len(batch_outputs)):
      batch_outputs[k] = batch_outputs[k][0]['generated_text'][len(prompts[j]):]
      j += 1
    out.extend(batch_outputs)


  res = []
  with open("predicted_intents.jsonl", "w", encoding="utf-8") as f:
    for conv_id, text in zip(conv_ids, out):
      intent_match = re.search(r'Intent:\s*(.+)', text)
      intent = intent_match.group(1).strip() if intent_match else "Couldn't find"

      rationale_match = re.search(r'Rationale:\s*(.+)', text)
      if rationale_match:
          rationale = rationale_match.group(1).split('.')[0] + '.'
      else:
          rationale = "Couldn't find"

      ans = {
          "conversation_id": conv_id,
          "predicted_intent": intent,
          "rationale": rationale
      }

      res.append(ans)
      f.write(json.dumps(ans, ensure_ascii=False) + "\n")

  print("\n",res)
  with open("predicted_intents.json", "w", encoding="utf-8") as f:
    json.dump(res, f, ensure_ascii=False, indent=2)

  df = pd.DataFrame(res)
  df.to_csv("predicted_intents.csv", index=False)


In [ ]:
from google.colab import files
import json

uploaded = files.upload()

filename = list(uploaded.keys())[0]

with open(filename, "r", encoding="utf-8") as f:
    conversations = json.load(f)
input_file = "conversations.json"

predictor(input_file)
print("JSON file is saved as predicted_intents.json")
print(f"CSV file is saved as predicted_intents.csv")


Saving conversations.json to conversations (2).json


100%|██████████| 1/1 [00:12<00:00, 12.23s/it]


 [{'conversation_id': 'conv_001', 'predicted_intent': 'Book Appointment', 'rationale': 'The user is inquiring about viewing properties.'}, {'conversation_id': 'conv_002', 'predicted_intent': 'Product Inquiry', 'rationale': 'User is asking about parking availability which is a feature of the apartment.'}, {'conversation_id': 'conv_003', 'predicted_intent': 'Pricing Negotiation', 'rationale': 'The user is asking for a price reduction.'}, {'conversation_id': 'conv_004', 'predicted_intent': 'Support Request', 'rationale': 'The user is experiencing a problem with accessing property documents, suggesting a support request.'}, {'conversation_id': 'conv_005', 'predicted_intent': 'Follow-Up', 'rationale': 'The user expressed interest but did not make a decision yet.'}]
